In [1]:
import cv2
import numpy as np
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# Callback function for trackbar changes (empty function for now)
def nothing(x):
    pass

# Function to track object from the device's camera with trackbars
def track_object_from_camera():
    cap = cv2.VideoCapture(0)  # 0 corresponds to the default camera (you can change it if you have multiple cameras)

    # Create a window for trackbars
    cv2.namedWindow('Trackbars')

    # Create trackbars for HSV range
    cv2.createTrackbar('H_min', 'Trackbars', 0, 255, nothing)
    cv2.createTrackbar('S_min', 'Trackbars', 0, 255, nothing)
    cv2.createTrackbar('V_min', 'Trackbars', 0, 255, nothing)

    cv2.createTrackbar('H_max', 'Trackbars', 255, 255, nothing)
    cv2.createTrackbar('S_max', 'Trackbars', 255, 255, nothing)
    cv2.createTrackbar('V_max', 'Trackbars', 255, 255, nothing)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Get current trackbar positions
        h_min = cv2.getTrackbarPos('H_min', 'Trackbars')
        s_min = cv2.getTrackbarPos('S_min', 'Trackbars')
        v_min = cv2.getTrackbarPos('V_min', 'Trackbars')

        h_max = cv2.getTrackbarPos('H_max', 'Trackbars')
        s_max = cv2.getTrackbarPos('S_max', 'Trackbars')
        v_max = cv2.getTrackbarPos('V_max', 'Trackbars')

        # Define the lower and upper HSV range
        min_color = np.array([h_min, s_min, v_min])
        max_color = np.array([h_max, s_max, v_max])

        # Create a mask using the target color range
        mask = cv2.inRange(hsv, min_color, max_color)

        # Bitwise-AND mask and original image
        result = cv2.bitwise_and(frame, frame, mask=mask)

        # Display the result
        cv2.imshow('Original', frame)
        cv2.imshow('Mask', mask)
        cv2.imshow('Result', result)

        if cv2.waitKey(1) & 0xFF == 27:  # Press 'Esc' to exit
            break

    cap.release()
    cv2.destroyAllWindows()

# Call the function to start capturing video from the camera
track_object_from_camera()


In [ ]:
import cv2
import numpy as np

# Function to capture and store the background
def capture_background():
    # Open the webcam
    cap = cv2.VideoCapture(0)

    # Allow the webcam to stabilize
    for _ in range(30):
        _, background = cap.read()

    # Flip the background horizontally for later overlay
    background = cv2.flip(background, 1)

    cv2.imwrite("E:/background.jpg", background)

    cap.release()

capture_background()


In [2]:
import cv2
import numpy as np

def invisible_cloak():
    background = cv2.imread("E:/background.jpg")

    cap = cv2.VideoCapture(0)

    while True:
        _, frame = cap.read()

        frame = cv2.flip(frame, 1)

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        lower_red = np.array([102, 51, 51])
        upper_red = np.array([153, 255, 255])

        mask = cv2.inRange(hsv, lower_red, upper_red)

        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((5, 5), np.uint8))
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, np.ones((5, 5), np.uint8))

        cloak = cv2.bitwise_not(mask)

        res1 = cv2.bitwise_and(frame, frame, mask=cloak)

        res2 = cv2.bitwise_and(background, background, mask=mask)

        final_output = cv2.addWeighted(res1, 1, res2, 1, 0)

        cv2.imshow('Invisible Cloak', final_output)

        if cv2.waitKey(1) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

invisible_cloak()
